<h1>US Domestic Airline Flights Performance Analysis</h1>

## Summary
This project contains flight delay analysis for US Domestic Airlines from 2010 to 2020 in a dashboard. This dashboard includes:
<ul>
<li>Monthly average carrier delay by reporting airline for the given year.</li>
<li>Monthly average weather delay by reporting airline for the given year.</li>
<li>Monthly average national air system delay by reporting airline for the given year.</li>
<li>Monthly average security delay by reporting airline for the given year.</li>
<li>Monthly average late aircraft delay by reporting airline for the given year.</li>
</ul>
<hr>

We first install required packages and import required libraries.

In [ ]:
# Install required packages
!pip install packaging
!pip install pandas dash
!pip install httpx==0.20 dash plotly

# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import html
from dash import dcc
from dash import Input
from dash import Output

Then, we create a dash application.

In [ ]:
# Create a dash application
app = dash.Dash(__name__)


We read the data into a pandas data frame.

In [ ]:
# Read the airline data into pandas data frame
airline_data = pd.read_csv('/kaggle/input/airline-data-for-ibm-coursera-data-sci-pro-cert/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

Now, we create our application layout. Our dashboard application layout has three components:<br>
<ul>
<li>Title of the application</li>
<li>Component to enter the year as an input inside a layout division</li>
<li>5 Charts conveying the different types of flight delay</li>
</ul>

In [ ]:
# Build application layout
app.layout = html.Div(children=[ html.H1('Flight Delay Time Statistics', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 30}),
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', 
                                type='number', min = "2010', max = "2020" style={'height':'35px', 
                                                                    'font-size': 30}),], 
                                style={'font-size': 30}),
                                html.Br(),
                                html.Br(), 
                                # Segment 1
                                html.Div([
                                        html.Div(dcc.Graph(id='carrier-plot')),
                                        html.Div(dcc.Graph(id='weather-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 2
                                html.Div([
                                        html.Div(dcc.Graph(id='nas-plot')),
                                        html.Div(dcc.Graph(id='security-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 3
                                html.Div(dcc.Graph(id='late-plot'), style={'width':'65%'})
                                ])

<p style='text-align: justify;'>Now, we define a function to take in airline data and selected year as an input and perform computation for creating charts and plots. This function returns computed average data frames for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.</p>

In [ ]:
def compute_info(airline_data, entered_year):
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late


<p style='text-align: justify;'>We define the callback function that uses the input provided to perform the computation. Then, we return the graphs with <code>get_graph()</code> function.</p>

In [ ]:
@app.callback( [
               Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'),
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')
               ],
               Input(component_id='input-year', component_property='value'))

# Function to add computation to the callback function and return the graphs
def get_graph(entered_year):
    
    # Compute required information for creating graphs from the data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)
            
    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline',
                          title='Average carrrier delay time (minutes) by airline')
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline',
                          title='Average weather delay time (minutes) by airline')
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline',
                      title='Average NAS delay time (minutes) by airline')
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline',
                      title='Average security delay time (minutes) by airline')
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline',
                       title='Average late aircraft delay time (minutes) by airline')
            
    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]

Finally, we run the application.

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()